#### Monte Carlo Simulation

In [1]:
import numpy as np   
import pandas as pd    
import random  as rd

In [2]:
 class Die:
    '''
    PURPOSE:
    A class accepts variety of random variables associated with stochastic
    processes and rolls/runs and returns a list of outcomes
    
    ATTRIBUTES:
    Takes an array of faces 
    
    METHODS:
    __init__::Return a dataframe with faces and weights column
    change_weight:: Changes the weight of a single side/face.
    roll_die:: Roll/Run the die one or more times
    show_state:: Display the current set of the faces and weights
    -------------------------------------------------------------------------
    '''
    def __init__(self , faces):
        '''
        PURPOSE:
        Initializes private dataframe containing faces and weights respectively
    
        INPUTS:
        Takes an array of faces 
    
        OUTPUTS:
        Return a dataframe with faces and weights column
        '''
        self.faces = list(set(faces)) # The faces must be unique
        self.weights= np.ones(len(faces))  # Initialize the weight to 1.0
        self.faces_weights_df= pd.DataFrame(self.faces , columns=['faces'])
        self.faces_weights_df = self.faces_weights_df.assign(weights=self.weights)
    def change_weight(self , face_value , new_weight):
        # face passed must be valid
        is_face_valid = (lambda face_value: face_value in self.faces) 
        if(not is_face_valid(face_value)): return "Error:The face passed is invalid."
        # Weight passed must be valid
        is_weight_valid = isinstance(new_weight, float) | isinstance(new_weight, int) | isinstance(new_weight, bool)
        if(not is_weight_valid): return "Error:The Weight passed is invalid."
        self.faces_weights_df.iloc[self.faces.index(face_value) , [1]] = new_weight
    def roll_die(self , number_of_rolls = 1):
         return [rd.choice(self.faces) for roll in range(number_of_rolls)]
    def show_state(self):
        return self.faces_weights_df

In [3]:
number_die_array = [1, 2 ,3 ,4 ,5 ,6]
string_coin_array = ['Heads', 'Tails']
d = s = Die(number_die_array)
d.show_state()
d.change_weight(2, 2.0)
d.change_weight(3, 3)
d.change_weight(4, 4)
d.change_weight(5, 5)
d.change_weight(6, 6)
d.roll_die(10)

[4, 4, 2, 4, 2, 5, 4, 1, 6, 6]

In [4]:
Die.__doc__

'\n   PURPOSE:\n   A class accepts variety of random variables associated with stochastic\n   processes and rolls/runs and returns a list of outcomes\n   \n   ATTRIBUTES:\n   Takes an array of faces \n   \n   METHODS:\n   __init__::Return a dataframe with faces and weights column\n   change_weight:: Changes the weight of a single side/face.\n   roll_die:: Roll/Run the die one or more times\n   show_state:: Display the current set of the faces and weights\n   -------------------------------------------------------------------------\n   '

In [5]:
class Game: # Or access using Game(Die)
    def __init__(self , dice):
        self.dice = dice  # Taking the list of dice
    def play(self, number_of_rolls):
        self.number_of_rolls = number_of_rolls
        self.play_result_df_list = []
        # Iterate through the list of dice and perform the same operations 
        for roll in self.dice:
            self.cols = [str(die) for die in roll.faces]
            roll_results = [roll for roll in roll.roll_die(self.number_of_rolls)] # Roll the dice X number of times extract the results  
            # Create a single dim array with similar results populate across the array
            roll_data = np.array([np.full((1 ,len(roll.faces)), roll_result)[0] for roll_result in roll_results]) 
            index = [i for i in range(1, len(roll_data) + 1)]
            self.play_df = pd.DataFrame(roll_data ,
                                   index = index,
                                   columns = self.cols)  
            self.play_df.index.name = 'roll number' 
            self.show(self.play_df , 1)
        
    
    def show(self , play_result_df , df_form = 1): # default df return form is Wide = 1 and Narrow option is 2
        if (df_form != 1 and df_form != 2):
            return print("Error:The dataframe display format option can only be Wide(value 1) or Narrow(value 2).")
        
        self.play_result_df_list.append(play_result_df)
        if df_form == 1:
            return play_result_df
        else:
            return pd.melt(panda_df, value_vars=self.cols, var_name='VARS', value_name='VALUES', ignore_index=False)
        
dice = []
dice.extend([s , d]) # Playing a Two dice 
g = Game(dice)
g.play(10)

In [30]:
# An analyzer takes the results of a single game and computes statistical properties about it.
class Analyzer:
     def __init__(self , game):
            self.game = game
            self.df_data_type = type(game.dice[0].faces[1]) # infers the data type of the die faces  
#             game.play(10)
            self.game_result = game.play_result_df_list
            
     def face_count(self):  
#         self.face_array = np.zeros(len(self.game.cols), dtype = int) #  Place out here to get the total 
#         Iterate through the resulting list array and extract each parallel value to match and increment properly
        for parallel_index in range(self.game.number_of_rolls):
            start = 0
            self.face_array = np.zeros(len(self.game.cols), dtype = int) # Hold the face value and increment when new similar value is found uses Game Cols Structure Copy
            while start < len(self.game_result):
                print(f"VAL IN {self.game_result[start].iloc[parallel_index]['1']}")
                self.face_array[self.game.cols.index(str(self.game_result[start].iloc[parallel_index]['1']))] = self.face_array[self.game.cols.index(str(self.game_result[start].iloc[parallel_index]['1']))] + 1
                start += 1
                
            print(self.face_array) 
        index = [i for i in range(1, len(self.face_array) + 1)]
        face_count_df = pd.DataFrame(self.face_array ,
                               index = index,
                               columns = self.game.cols)  
            

a = Analyzer(g)
a.face_count()

VAL IN 4
VAL IN 6
[0 0 0 1 0 1]
VAL IN 5
VAL IN 6
[0 0 0 0 1 1]
VAL IN 2
VAL IN 1
[1 1 0 0 0 0]
VAL IN 6
VAL IN 5
[0 0 0 0 1 1]
VAL IN 2
VAL IN 2
[0 2 0 0 0 0]
VAL IN 5
VAL IN 3
[0 0 1 0 1 0]
VAL IN 3
VAL IN 6
[0 0 1 0 0 1]
VAL IN 3
VAL IN 5
[0 0 1 0 1 0]
VAL IN 4
VAL IN 5
[0 0 0 1 1 0]
VAL IN 5
VAL IN 5
[0 0 0 0 2 0]


ValueError: Shape of passed values is (6, 1), indices imply (6, 6)

In [ ]:
["foo", "bar", "baz"].index("bar")

In [ ]:
# ['1','2','3','4','5','6'].index('3') 

In [ ]:
for i in  range(5):
    print(i)